In [ ]:
from os.path import join, basename
import os
import numpy as np
import pandas as pd
import xarray as xr
# spatial libraries
import geopandas as gp 

from  shapely.geometry import Point
import rasterio
import rasterio.transform
# flow direction library
import pyflwdir # https://gitlab.com/deltares/wflow/pyflwdir

In [ ]:
# function to read values from spatial raster
def sample_map(x, y, fn_map, layer=1):
    """read map values at x, y coordinates from file"""
    if not os.path.isfile(fn_map): 
        raise IOError("{} file not found".format(fn_map))
    with rasterio.open(fn_map, 'r') as src:
        # assume low resolution lat lon coordinates are given
        r, c = src.index(x, y)
        r, c = np.atleast_1d(r).astype(int), np.atleast_1d(c).astype(int)
        nrows, ncols = src.shape
        valid = np.logical_and.reduce((r>=0, r<nrows, c>=0, c<ncols))
        sample = np.ones(r.size, dtype=src.dtypes[layer-1])*np.nan
        sample[valid] = src.read(layer)[r[valid], c[valid]]
    return sample

def sample_map_mem(x, y, map_data, transform):
    """read map_data (np.ndarray) values at x, y coordinates"""
    r, c = rasterio.transform.rowcol(transform, x, y)
    r, c = np.atleast_1d(r).astype(int), np.atleast_1d(c).astype(int)
    nrows, ncols = map_data.shape
    valid = np.logical_and.reduce((r>=0, r<nrows, c>=0, c<ncols))
    sample = np.ones(r.size, dtype=map_data.dtype)*np.nan
    sample[valid] = map_data[r[valid], c[valid]]
    return sample

In [ ]:
# read some cama-flood maps and parse flow direction data

map_dir = r'/home/dirk/models/cama-flood_bmi_v3.6.2_nc/map/global_15min'

fn_nextxy = join(map_dir, 'nextxy.tif')
with rasterio.open(fn_nextxy, 'r') as src:
    nextxy = src.read()
    transform = src.transform
    flw = pyflwdir.FlwdirRaster(data=nextxy, ftype='flow')


In [ ]:
# accumulated ucat worldpop data over flow direction
fn_exp_agg = join(map_dir, 'worldpop.tif')
with rasterio.open(fn_exp_agg, 'r') as src:
    exp_agg = src.read(1)
    with rasterio.open(fn_exp_agg.replace('.tif', '_accu.tif'), 'w', **src.profile) as dst:
        exp_agg_accu = flw.accuflux(exp_agg)
        dst.write(exp_agg_accu.astype(np.float32), 1)

fn_exp_agg_lecz = join(map_dir, 'lecz_worldpop.tif')
with rasterio.open(fn_exp_agg_lecz, 'r') as src:
    exp_agg_lecz = src.read(1)     
    with rasterio.open(fn_exp_agg_lecz.replace('.tif', '_accu.tif'), 'w', **src.profile) as dst:
        exp_agg_lecz_accu = flw.accuflux(exp_agg_lecz)
        dst.write(exp_agg_lecz_accu.astype(np.float32), 1)

In [ ]:
# combine simulations
ddir = join(root, 'data', '3-model_output')
scens = ['cmpnd', 'runoff']
mods = ['anu', 'cnrs', 'ecmwf', 'jrc', 'nerc']
rp=10
mod_lst = []
for mod in mods:
    scen_lst = []
    for scen in scens:
        fn_swe = join(ddir, f'ev_map_sfcelv_{mod}_mswep_{scen}_v362_1980-2014.nc')
        scen_lst.append(xr.open_dataset(fn_swe, chunks={'lat':360, 'lon':360})['sfcelv_ev'])
    mod_lst.append(xr.concat(scen_lst, dim='scen'))
ds = xr.concat(mod_lst, dim='ensemble')
ds['ensemble'] = xr.Variable('ensemble', mods)
ds['scen'] = xr.Variable('scen', scens)
ds_diff = (ds.sel(T=rp, scen='cmpnd')-ds.sel(T=rp, scen='runoff')).compute()

In [ ]:
# calculated people affected by dH

import warnings
warnings.simplefilter('ignore')
coupling = pd.read_csv(fn_csv_coupling2, index_col='index')
lon, lat, index = coupling['cmf_lon_15min'], coupling['cmf_lat_15min'], coupling.index

# read total exposure
fn_exp_agg = join(map_dir, 'worldpop_accu.tif')
fn_exp_agg_lecz = join(map_dir, 'lecz_worldpop_accu.tif')
pop_all_rivmth = sample_map(lon, lat, fn_exp_agg)
pop_lecz_rivmth = sample_map(lon, lat, fn_exp_agg_lecz)


rps =[2, 10, 50, 100]
mod_lst = []
for mod in mods:
    rp_lst =[]
    for rp in rps:
        fn_tif = join(ddir, f'{mod}_cmpnd', f'worldpop_agg_T{rp:03d}.tif')
        
        with rasterio.open(fn_tif, 'r') as src:
            mv_mask = src.read(1)==src.nodata
            
            # accumulate total no. of people affected
            pop_affected_all_accu = flw.accuflux(src.read(1))
            pop_affected_all_accu[mv_mask] = src.nodata
            pop_affected_all_rivmth = sample_map_mem(lon, lat, pop_affected_all_accu, src.transform)
                
            # in flood plains with larger flood depth
            ds_diff = (ds.sel(T=rp, scen='cmpnd')-ds.sel(T=rp, scen='runoff')).compute()
            mask = ds_diff.sel(ensemble=mod).squeeze().values>0.01   
            pop_affected_dH = np.where(mask, src.read(1), 0)
            pop_affected_dH_accu = flw.accuflux(pop_affected_dH)
            pop_affected_dH_accu[mv_mask] = src.nodata
            pop_affected_dH_rivmth = sample_map_mem(lon, lat, pop_affected_dH_accu, src.transform)

            rp_lst.append(xr.merge([
                xr.DataArray(dims=('index'), data=pop_affected_dH_rivmth, coords=dict(index=index), name='people_affected_dH'),
                xr.DataArray(dims=('index'), data=pop_affected_all_rivmth, coords=dict(index=index), name='people_affected_all'),
            ]))
    mod_lst.append(xr.concat(rp_lst, dim='T'))
ds_out = xr.merge([
    xr.concat(mod_lst, dim='ensemble'),
    xr.DataArray(dims=('index'), data=pop_all_rivmth, coords=dict(index=index), name='people_all'),
    xr.DataArray(dims=('index'), data=pop_lecz_rivmth, coords=dict(index=index), name='people_lecz'),
])
ds_out['ensemble'] = xr.Variable('ensemble', mods)
ds_out['T'] = xr.Variable('T', rps)
ds_out.to_netcdf(join(ddir, 'rivmth_pop_exposed.nc'))

In [ ]:
#TODO: delete below?

In [ ]:
# exepcted annual mean people exposed
fn_impact = join(ddir, 'rivmth_pop_affected.nc')
ds_impact = xr.open_dataset(fn_impact)

T0 = xr.DataArray(
    dims=('ensemble', 'T', 'index'), 
    coords={'ensemble':ds_impact.ensemble, 'T': [1], 'index': ds_impact.index}, 
    data=np.zeros((ds_impact.ensemble.size,1,ds_impact.index.size))
)
pop_affected_dH_dp = xr.concat([T0, ds_impact['people_affected_dH']], dim='T')
pop_affected_dH_dp['p'] = xr.Variable('T', 1/pop_affected_dH_dp['T'].values)
pop_affected_dH_dp = pop_affected_dH_dp.sel(T=pop_affected_dH_dp['T'].values[::-1]).swap_dims({'T':'p'}) 
ds_impact['people_affected_dH_dp'] = pop_affected_dH_dp.integrate('p')
pop_affected_all_dp = xr.concat([T0, ds_impact['people_affected_all']], dim='T')
pop_affected_all_dp['p'] = xr.Variable('T', 1/pop_affected_all_dp['T'].values)
pop_affected_all_dp = pop_affected_all_dp.sel(T=pop_affected_all_dp['T'].values[::-1]).swap_dims({'T':'p'}) 
ds_impact['people_affected_all_dp'] = pop_affected_all_dp.integrate('p')
# ds_impact

In [ ]:
# compute relative people exposed
df = (ds_impact['people_affected_dH_dp'].sum('index')/ds_impact['people_affected_all_dp'].sum('index')*100).to_series()
df['mean'] = df.mean()
df

In [ ]:
## plot
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from plot_tools import *
import cartopy.crs as ccrs
import seaborn as sns
from string import ascii_uppercase as letters

fdir = join(root, 'reports', 'figures')
rc = {'savefig.bbox': 'tight',  'savefig.format': 'png', 'savefig.dpi':300}
context = 'paper'
# sns.set(context=context, style='whitegrid', font_scale=0.75 if context == 'talk' else 1., rc=rc)
sns.set(context=context, style='whitegrid', font_scale=1.2 if context == 'paper' else 1., rc=rc)
crs = ccrs.PlateCarree()

In [ ]:
T = 10
model = 'mean'
attrs_fn = join(ddir, 'rivmth_mean_attrs.csv')
attrs = pd.read_csv(attrs_fn, index_col='index').rename(columns={'rivmth_lat':'lat', 'rivmth_lon':'lon'}).drop(3354)

gdf = pandas2geopandas(pd.concat([
    attrs,
    ds_impact.mean('ensemble').sel(T=T).reset_coords(drop=True).to_dataframe()
], axis=1))
gdf['people_affected_dH_perc'] = gdf['people_affected_dH'] /  np.maximum(gdf['people_affected_all'],1) * 100
gdf['people_affected_dH_dp_perc'] = gdf['people_affected_dH_dp'] /  np.maximum(gdf['people_affected_all_dp'],1) * 100
gdf['people_affected_dH_dp_percLECZ'] = np.minimum(gdf['people_affected_dH_dp'] / np.maximum(gdf['people_lecz'],1) * 100, 100)
gdf['people_affected_dH_dp_percALL'] = gdf['people_affected_dH_dp'] /  np.maximum(gdf['people_all'],1) * 100

In [ ]:
basins = gpd.read_file(r'/home/dirk/models/cama-flood_bmi_v3.6.2/map/global_15min/flw_basins')
basins['area'] = basins.area
basins['rivmth_idx'] = basins['DN'] - 1
basins = basins.merge(gdf.drop(columns='geometry'), on='rivmth_idx', how='right')
# fig, ax = plt.subplots(1,1, figsize=(10,5))
# basins.plot(linewidth=0.1, ax=ax)

In [ ]:
column = f'people_affected_dH_dp'
label=f'people affected [-]'
vmin, vmax, n= 2, 6, 5
cticks=np.linspace(vmin, vmax, n)
cmap = ListedColormap(sns.cubehelix_palette(16, start=.5, rot=-.75))
plot_kwargs=dict(edgecolor=(0.5, 0.5, 0.5, 0.8), linewidth=0.1, legend=False, zorder=2)

gdf1 = basins.copy()
if 'perc' not in column:
    gdf1[column] = np.where(gdf1[column]<=0, 0, np.log10(gdf1[column]))
plt.close('all')
fig = plt.figure(figsize=(18, 10.5))
grid = plt.GridSpec(2, 1, wspace=0.01, hspace=0.01)

# map 1
axg = fig.add_subplot(grid[0, 0], projection=crs)
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=False, outline=False, features=['land'],)
cax = plot_choropleth(
    fig, axg, gdf1, column=column, 
    cmap=plt.cm.Greens , cticks=cticks, vmin=vmin, vmax=vmax, discrete=False, 
    plot_kwargs = plot_kwargs,
    cbar_kwargs = dict(label=label, location='right'),
    cbar_pos = dict(pad=0.02, fraction=0.01, shrink=0.5)
)
cax.set_yticklabels([f'$10^{e:0.0f}$' for e in cticks])
axg.text(-175, 85, letters[0]+'', transform=crs, fontsize='large')
    
column = f'people_affected_dH_dp_perc'
label=f'people affected [%]'
vmin, vmax, n= 0, 40, 5
cticks=np.linspace(vmin, vmax, n)
# cmap = ListedColormap(sns.cubehelix_palette(16, start=.5, rot=-.75))

# map 2
axg2 = fig.add_subplot(grid[1, 0], projection=crs)
basemap(axg2, bbox=(-180, -60, 180, 90), gridlines=False, outline=False, features=['land'],)
plot_kwargs.pop('marker',None)
plot_kwargs.pop('markersize',None)
plot_kwargs.update(linewidth=0.1)
cax2 = plot_choropleth(
    fig, axg2, gdf1, column=column, 
    cmap=plt.cm.Blues, cticks=cticks, vmin=vmin, vmax=vmax, discrete=False, 
    plot_kwargs = plot_kwargs,
    cbar_kwargs = dict(label=label, location='right'),
    cbar_pos = dict(pad=0.02, fraction=0.01, shrink=0.5)
) 
axg2.text(-175, 85, letters[1], transform=crs, fontsize='large')


fn_fig = join(fdir, f'expected_annual_people_exposed_{model}.png')
print(basename(fn_fig))
plt.savefig(fn_fig)